# Notebook imports

In [75]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from os import walk
from os.path import join
import nltk #package for NLP
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.model_selection import GridSearchCV 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error, r2_score

# Extract news from all xml files from all the directories
### Extracting the filename,  headline,  text,  biptopics, date published and itemid from every news article

In [2]:
def read_xml_files():
    list =[]
    count = 0
    dataframes = []
    for roots, dirs, files in walk('xml_files'):# code to read all the files in the given folders
        for file in files:
            if not file.startswith('.'): # to exclude the unwanted system files
                file_path = join(roots,file)
                tree = ET.parse(file_path) #parsing the.xml files
                root = tree.getroot() 
                head_line = root.find("headline") #Extracting the head_line
                text = "";    #Extracting body and text
                body = root.find("text") #Extracting the text body
                for data in body:
                    text = text + " " + data.text
                
                meta_data = root.find("metadata")
                codes = meta_data.findall("codes")
                code_array = []
                for i in codes:
                    if i.attrib['class']=='bip:topics:1.0':
                        code = i.findall("code") #Extracting the biptopics
                        for code_value in code:
                            code_array.append(code_value.attrib['code'])
                            
                dc_elements = meta_data.findall("dc")
                for dc in dc_elements:
                    if dc.attrib['element']=='dc.date.published': #Extracting the dc.date.published
                        date_published = dc.attrib['value']
                        
                item_id = root.attrib['itemid'] #Extracting the filename
                df = pd.DataFrame({"filename":file, "headline":head_line.text, "text":text, "biptopics":code_array, "dc_date_published":date_published, "itemid":item_id})
                dataframes.append(df)
                
    
    data_frame = pd.concat(dataframes,ignore_index=True) #creating a dataframe with all the extracted information for every news article
    return data_frame

In [3]:
df = read_xml_files() # initial dataframe with the original data fromall the .xml files

# Function to extract all the possible biptopics
### The function returns all the biptopics even if a news article has more than one.

In [4]:
def extract_biptopics(df):
    
    biptopics = set()
    
    for row in df['biptopics']:
        biptopics.add(row)
        
    return biptopics

In [5]:
biptopics = extract_biptopics(df) #Call the function to extract all the biptopics

{'C11',
 'C12',
 'C13',
 'C14',
 'C15',
 'C151',
 'C1511',
 'C152',
 'C16',
 'C17',
 'C171',
 'C172',
 'C173',
 'C174',
 'C18',
 'C181',
 'C182',
 'C183',
 'C21',
 'C22',
 'C23',
 'C24',
 'C31',
 'C311',
 'C312',
 'C313',
 'C32',
 'C33',
 'C331',
 'C34',
 'C41',
 'C411',
 'C42',
 'CCAT',
 'E11',
 'E12',
 'E121',
 'E13',
 'E131',
 'E132',
 'E14',
 'E141',
 'E142',
 'E143',
 'E21',
 'E211',
 'E212',
 'E31',
 'E311',
 'E312',
 'E313',
 'E41',
 'E411',
 'E51',
 'E511',
 'E512',
 'E513',
 'E61',
 'E71',
 'ECAT',
 'G15',
 'G151',
 'G152',
 'G153',
 'G154',
 'G155',
 'G156',
 'G157',
 'G158',
 'G159',
 'GCAT',
 'GCRIM',
 'GDEF',
 'GDIP',
 'GDIS',
 'GENT',
 'GENV',
 'GFAS',
 'GHEA',
 'GJOB',
 'GOBIT',
 'GODD',
 'GPOL',
 'GPRO',
 'GREL',
 'GSCI',
 'GSPO',
 'GTOUR',
 'GVIO',
 'GVOTE',
 'GWEA',
 'GWELF',
 'M11',
 'M12',
 'M13',
 'M131',
 'M132',
 'M14',
 'M141',
 'M142',
 'M143',
 'MCAT'}

# Removing duplicate biptopic for every document

### Note : I am working with single label for every data/news article hence keeping only the first label of every news article

In [7]:
# Reference : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html
data = df.drop_duplicates(subset ="filename", keep = 'first') #dataframe with unique documents
data.sort_index(inplace=True) # Sorting the indices of the df dataframe
data.reset_index(inplace=True)
data.drop(columns='index',inplace=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# Function to clean the text body
### remove html tags, tokenize, stemming, remove stopwords & remove non-alphabets

In [8]:
def text_processing(data):
    
    list_rows = []
    
    stop_words = set(stopwords.words('english')) #convert the stopwords to a set and store it in the variable
    stemmer = PorterStemmer() #variable to use porterstemmer object
        
    for row in data.text:
        
        #news_text = row['text']
        
        token_words=[]
        filtered_words = ""
        soup = BeautifulSoup(row,'html.parser') #html.parser is the type of parser used to remove html tags
        text = soup.get_text() #get the text from the BeautifulSoup object
        text_lower = text.lower() #convert the words to lowercase
        text_lower = text_lower.strip()
        token_words = (word_tokenize(text_lower)) #method to tokenize the message
        
        for word in token_words:
            if word not in stop_words and word.isalpha(): #remove/filter non-aphabets
                stemmed_word = stemmer.stem(word) #stemming to the root word
                filtered_words = filtered_words + " " + stemmed_word
                
        list_rows.append(filtered_words)
       
    
    return list_rows #return the list of all the cleaned news articles

# Function to extract all the features and convert to a dataframe and sparse matrix

In [14]:
def extract_features(data):

    new_list = text_processing(data) # call the text_processing function
    new_cleaned_data = data #copy the dataframe to a new one to modify with the cleaned text
    new_cleaned_data.text = new_list # replacing the original text with the cleaned, stemmed news text
    
    vectorizer2 = TfidfVectorizer(stop_words = 'english', strip_accents='unicode',use_idf=True)
    #to create the vocabulary and the document term matrix
    
    all_features = vectorizer2.fit_transform(new_cleaned_data.text) # to create a sparse matrix of the words in the text
    
    converted_df = pd.DataFrame(all_features.toarray(), columns = vectorizer2.get_feature_names())#convert the sparse matrix to a dataframe
    converted_df['labels_biptopics'] = data.biptopics
    
    return all_features,converted_df
    

In [15]:
all_features,converted_df = extract_features(data) #call the above function

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [17]:
converted_df # the sparse matrix of all the features

,aa,aaa,aaal,aachen,aad,aadn,aag,aagev,aah,aaholm,...,zyprexa,zypro,zyrtec,zysblat,zyuganov,zywcu,zywiec,zywnosciowej,zzn,labels_biptopics
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCAT
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCAT
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GDEF
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCAT
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GCAT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C15
48253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,E11
48254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C11
48255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C31


# Create Train and Test data set into features and labels
## Reason to use train test split : 

#### The train-test split is normally done using a 70:30 ratio of train data to the test data out of the entire dataset. 

#### Since the amount of data given to train and test the model was very huge, it was appropriate to take the train test split approach as it trains on the 70% dataset and the test on the 30% dataset. 

#### On the other hand, cross validation iteratively trains on shuffled sets of train and test datasets. 

#### However, it will run for longer time periods as there is huge amount of data and computational constraints. 

#### Hence, I prefered test train split over the other methods.

In [18]:
y = data['biptopics'] #labes
x = all_features #labels
X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size = 0.3,random_state=100) #split the features and labels into train and test data

In [78]:
# The below function trains the model from the dataframe. 
#However, I was not able to complete the training with the below function as I was facing continuos kernel dead issues when I was passing the dataframe.
#As a result of which I have used the next function train_classifiers()

def train_classifiers_data_frame(classifier_type,data_frame,p={}):
    bip_topics = data_frame['labels_biptopics']
    new_data_frame = data_frame.drop(columns=['labels_biptopics'])
    X_train,X_test,Y_train,Y_test = train_test_split(new_data_frame,bip_topics,test_size = 0.3,random_state=100) #split the dataframe into train and test data
    if classifier_type == 'MNB': 
        MNB_classifier = MultinomialNB()
        MNB_classifier.set_params(**p)
        
        #training the naive bayes model
        MNB_classifier.fit(X_train, Y_train) 
        return MNB_classifier
    
    #create a Random Forest model
    elif classifier_type == 'RF': 
        RF_classifier = RandomForestClassifier()
        RF_classifier.set_params(**p)
        
        #training the Random Forest model
        RF_classifier.fit(X_train, Y_train) 
        return RF_classifier
    
    #create a Neural Network model
    elif classifier_type == 'NN': 
        NN_classifier = MLPClassifier()
        NN_classifier.set_params(**p)
        
        #training the Neural Network model
        NN_classifier.fit(X_train, Y_train) 
        return NN_classifier
    
    #create a Decision Tree classifier model
    elif classifier_type == 'DT': 
        DT_classifier = DecisionTreeClassifier()
        DT_classifier.set_params(**p)
        
        #training the Decision Tree model
        DT_classifier.fit(X_train, Y_train) 
        return DT_classifier
    
    #create a SVM SVC classifier model
    elif classifier_type == 'SVC': 
        SVC_classifier = svm.SVC()
        SVC_classifier.set_params(**p)
        
        #training the SVM SVC model
        SVC_classifier.fit(X_train, Y_train) 
        return SVC_classifier
    
    
    elif classifier_type == 'LR': 
        linear_regression = LinearRegression()
        #training the LinearRegression model
        linear_regression.fit(X_train, Y_train) 
        return linear_regression

# Function to train the classifiers

In [70]:
# I have used this function to train all the classifiers

def train_classifiers(X_train,Y_train,classifier_type,p={}):
    
    #create a naive bayes model
    if classifier_type == 'MNB': 
        MNB_classifier = MultinomialNB()
        MNB_classifier.set_params(**p)
        
        #training the naive bayes model
        MNB_classifier.fit(X_train, Y_train) 
        return MNB_classifier
    
    #create a Random Forest model
    elif classifier_type == 'RF': 
        RF_classifier = RandomForestClassifier()
        RF_classifier.set_params(**p)
        
        #training the Random Forest model
        RF_classifier.fit(X_train, Y_train) 
        return RF_classifier
    
    #create a Neural Network model
    elif classifier_type == 'NN': 
        NN_classifier = MLPClassifier()
        NN_classifier.set_params(**p)
        
        #training the Neural Network model
        NN_classifier.fit(X_train, Y_train) 
        return NN_classifier
    
    #create a Decision Tree classifier model
    elif classifier_type == 'DT': 
        DT_classifier = DecisionTreeClassifier()
        DT_classifier.set_params(**p)
        
        #training the Decision Tree model
        DT_classifier.fit(X_train, Y_train) 
        return DT_classifier
    
    #create a SVM SVC classifier model
    elif classifier_type == 'SVC': 
        SVC_classifier = svm.SVC()
        SVC_classifier.set_params(**p)
        
        #training the SVM SVC model
        SVC_classifier.fit(X_train, Y_train) 
        return SVC_classifier
    
    
    elif classifier_type == 'LR': 
        linear_regression = LinearRegression()
        #training the LinearRegression model
        linear_regression.fit(X_train, Y_train) 
        return linear_regression

# Function to calculate the F1 score of all the classifiers

## Reason to use F1 Score to measure the performace of the classifier : 

#### F1 Score is the weighted average of Precision and Recall.  

#### Hence, the F1 score considers both false positives and false negatives. 

#### F1 Score parameter is calculated by taking into account both the ratio of the correct positive predictions to the total positive predictions and the ratio of correctly predicted positive predictions to the total predictions. 

#### As it considers both precision and recall, I preferred the F1 Score as it gives a more accurate prediction performance of a classifier.




In [72]:
def calculate_f1score(classifier, classifier_type,X_test,Y_test):
    
    #calculate the F1 score of Naive Bayes model
    if classifier_type == 'MNB': 
        return f1_score(Y_test, classifier.predict(X_test), average='weighted')
    
    
    #calculate the F1 score of Random Forest model
    elif classifier_type == 'RF': 
        return f1_score(Y_test, classifier.predict(X_test), average='weighted')
    
    
    #calculate the F1 score of Neural Network model
    elif classifier_type == 'NN': 
        return f1_score(Y_test, classifier.predict(X_test), average='weighted')
    
    
    #calculate the F1 score of Decision Tree model
    elif classifier_type == 'DT': 
        return f1_score(Y_test, classifier.predict(X_test), average='weighted')
    
    
    #calculate the F1 score of SVM SVC model
    elif classifier_type == 'SVC': 
        return f1_score(Y_test, classifier.predict(X_test), average='weighted')
    

## Performance of MNB Classifier Before Tuning

In [31]:
MNB_classifier = train_classifiers(X_train,Y_train,'MNB') 

In [32]:
f1_score_MNB = calculate_f1score(MNB_classifier, 'MNB',X_test,Y_test) 
f1_score_MNB

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.40928166669439386

## Performance of MNB Classifier After Tuning

In [35]:
MNB_classifier = train_classifiers(X_train,Y_train,'MNB',{"alpha":0.0001})

In [36]:
f1_score_MNB = calculate_f1score(MNB_classifier, 'MNB',X_test,Y_test)
f1_score_MNB

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.7113232971484191

## Performance of Random Forest Classifier Before Tuning

In [37]:
RF_classifier = RandomForestClassifier()
RF_classifier = train_classifiers(X_train,Y_train,'RF',{'n_jobs' : -1, 'verbose' : 2})

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 10
building tree 2 of 10building tree 3 of 10
building tree 4 of 10

building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.0s finished


In [38]:
f1_score_RF = calculate_f1score(RF_classifier, 'RF',X_test,Y_test)
f1_score_RF

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.1s finished
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.6443379634355636

# Tuning the Random Forest Classifier

In [168]:
# I have run the gridsearch tuning in another jupyter notebook as a part of testing to get the gridsearch results
from sklearn.model_selection import GridSearchCV# Create the parameter grid based on the results of random search 
 
n_estimators = [600, 800, 1000] #no of trees used in the Random Forest
bootstrap = [True, False]

param_tune = {'n_estimators':n_estimators, 'bootstrap':bootstrap}

rf = RandomForestClassifier()# Instantiate the grid search model

grid_search = GridSearchCV(estimator = rf, param_grid = param_tune, 
                          cv = 5, n_jobs = -1, verbose = 2)

# Fit the grid search to the data
grid_search.fit(x_train, y_train)


Fitting 5 folds for each of 8 candidates, totalling 40 fits


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 93.7min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 133.6min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [170]:
grid_search.best_params_

{'bootstrap': False, 'n_estimators': 1000}

## Performance of Random Forest Classifier After Tuning

In [40]:
RF_classifier = RandomForestClassifier()
RF_classifier = train_classifiers(X_train,Y_train,'RF',{'n_estimators':1000, 'bootstrap':False,'n_jobs' : -1, 'verbose' : 20})  #call the function to train the Random forest classifier

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 1000building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000



[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    6.9s


building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   13.8s


...

[Parallel(n_jobs=-1)]: Done 992 tasks      | elapsed: 32.0min


building tree 996 of 1000


[Parallel(n_jobs=-1)]: Done 993 tasks      | elapsed: 32.0min


building tree 997 of 1000
building tree 998 of 1000
building tree 999 of 1000
building tree 1000 of 1000


[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 32.2min finished


In [41]:
f1_score_RF = calculate_f1score(RF_classifier, 'RF',X_test,Y_test)
f1_score_RF

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done  15 tasks      | elapsed:    0.4s
[Para

[Parallel(n_jobs=4)]: Done 134 tasks      | elapsed:    3.1s
[Parallel(n_jobs=4)]: Done 135 tasks      | elapsed:    3.1s
[Parallel(n_jobs=4)]: Done 136 tasks      | elapsed:    3.1s
[Parallel(n_jobs=4)]: Done 137 tasks      | elapsed:    3.2s
[Parallel(n_jobs=4)]: Done 138 tasks      | elapsed:    3.2s
[Parallel(n_jobs=4)]: Done 139 tasks      | elapsed:    3.2s
[Parallel(n_jobs=4)]: Done 140 tasks      | elapsed:    3.2s
[Parallel(n_jobs=4)]: Done 141 tasks      | elapsed:    3.2s
[Parallel(n_jobs=4)]: Done 142 tasks      | elapsed:    3.2s
[Parallel(n_jobs=4)]: Done 143 tasks      | elapsed:    3.3s
[Parallel(n_jobs=4)]: Done 144 tasks      | elapsed:    3.3s
[Parallel(n_jobs=4)]: Done 145 tasks      | elapsed:    3.3s
[Parallel(n_jobs=4)]: Done 146 tasks      | elapsed:    3.3s
[Parallel(n_jobs=4)]: Done 147 tasks      | elapsed:    3.4s
[Parallel(n_jobs=4)]: Done 148 tasks      | elapsed:    3.4s
[Parallel(n_jobs=4)]: Done 149 tasks      | elapsed:    3.4s
[Parallel(n_jobs=4)]: Do

[Parallel(n_jobs=4)]: Done 272 tasks      | elapsed:    7.1s
[Parallel(n_jobs=4)]: Done 273 tasks      | elapsed:    7.1s
[Parallel(n_jobs=4)]: Done 274 tasks      | elapsed:    7.1s
[Parallel(n_jobs=4)]: Done 275 tasks      | elapsed:    7.2s
[Parallel(n_jobs=4)]: Done 276 tasks      | elapsed:    7.2s
[Parallel(n_jobs=4)]: Done 277 tasks      | elapsed:    7.2s
[Parallel(n_jobs=4)]: Done 278 tasks      | elapsed:    7.2s
[Parallel(n_jobs=4)]: Done 279 tasks      | elapsed:    7.2s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    7.3s
[Parallel(n_jobs=4)]: Done 281 tasks      | elapsed:    7.3s
[Parallel(n_jobs=4)]: Done 282 tasks      | elapsed:    7.4s
[Parallel(n_jobs=4)]: Done 283 tasks      | elapsed:    7.4s
[Parallel(n_jobs=4)]: Done 284 tasks      | elapsed:    7.5s
[Parallel(n_jobs=4)]: Done 285 tasks      | elapsed:    7.5s
[Parallel(n_jobs=4)]: Done 286 tasks      | elapsed:    7.5s
[Parallel(n_jobs=4)]: Done 287 tasks      | elapsed:    7.6s
[Parallel(n_jobs=4)]: Do

[Parallel(n_jobs=4)]: Done 408 tasks      | elapsed:   11.1s
[Parallel(n_jobs=4)]: Done 409 tasks      | elapsed:   11.1s
[Parallel(n_jobs=4)]: Done 410 tasks      | elapsed:   11.1s
[Parallel(n_jobs=4)]: Done 411 tasks      | elapsed:   11.2s
[Parallel(n_jobs=4)]: Done 412 tasks      | elapsed:   11.2s
[Parallel(n_jobs=4)]: Done 413 tasks      | elapsed:   11.3s
[Parallel(n_jobs=4)]: Done 414 tasks      | elapsed:   11.3s
[Parallel(n_jobs=4)]: Done 415 tasks      | elapsed:   11.3s
[Parallel(n_jobs=4)]: Done 416 tasks      | elapsed:   11.3s
[Parallel(n_jobs=4)]: Done 417 tasks      | elapsed:   11.3s
[Parallel(n_jobs=4)]: Done 418 tasks      | elapsed:   11.4s
[Parallel(n_jobs=4)]: Done 419 tasks      | elapsed:   11.4s
[Parallel(n_jobs=4)]: Done 420 tasks      | elapsed:   11.4s
[Parallel(n_jobs=4)]: Done 421 tasks      | elapsed:   11.5s
[Parallel(n_jobs=4)]: Done 422 tasks      | elapsed:   11.5s
[Parallel(n_jobs=4)]: Done 423 tasks      | elapsed:   11.6s
[Parallel(n_jobs=4)]: Do

[Parallel(n_jobs=4)]: Done 549 tasks      | elapsed:   15.1s
[Parallel(n_jobs=4)]: Done 550 tasks      | elapsed:   15.1s
[Parallel(n_jobs=4)]: Done 551 tasks      | elapsed:   15.1s
[Parallel(n_jobs=4)]: Done 552 tasks      | elapsed:   15.2s
[Parallel(n_jobs=4)]: Done 553 tasks      | elapsed:   15.2s
[Parallel(n_jobs=4)]: Done 554 tasks      | elapsed:   15.2s
[Parallel(n_jobs=4)]: Done 555 tasks      | elapsed:   15.2s
[Parallel(n_jobs=4)]: Done 556 tasks      | elapsed:   15.2s
[Parallel(n_jobs=4)]: Done 557 tasks      | elapsed:   15.3s
[Parallel(n_jobs=4)]: Done 558 tasks      | elapsed:   15.3s
[Parallel(n_jobs=4)]: Done 559 tasks      | elapsed:   15.3s
[Parallel(n_jobs=4)]: Done 560 tasks      | elapsed:   15.3s
[Parallel(n_jobs=4)]: Done 561 tasks      | elapsed:   15.4s
[Parallel(n_jobs=4)]: Done 562 tasks      | elapsed:   15.4s
[Parallel(n_jobs=4)]: Done 563 tasks      | elapsed:   15.4s
[Parallel(n_jobs=4)]: Done 564 tasks      | elapsed:   15.5s
[Parallel(n_jobs=4)]: Do

[Parallel(n_jobs=4)]: Done 690 tasks      | elapsed:   19.1s
[Parallel(n_jobs=4)]: Done 691 tasks      | elapsed:   19.1s
[Parallel(n_jobs=4)]: Done 692 tasks      | elapsed:   19.1s
[Parallel(n_jobs=4)]: Done 693 tasks      | elapsed:   19.2s
[Parallel(n_jobs=4)]: Done 694 tasks      | elapsed:   19.2s
[Parallel(n_jobs=4)]: Done 695 tasks      | elapsed:   19.2s
[Parallel(n_jobs=4)]: Done 696 tasks      | elapsed:   19.2s
[Parallel(n_jobs=4)]: Done 697 tasks      | elapsed:   19.3s
[Parallel(n_jobs=4)]: Done 698 tasks      | elapsed:   19.3s
[Parallel(n_jobs=4)]: Done 699 tasks      | elapsed:   19.3s
[Parallel(n_jobs=4)]: Done 700 tasks      | elapsed:   19.3s
[Parallel(n_jobs=4)]: Done 701 tasks      | elapsed:   19.4s
[Parallel(n_jobs=4)]: Done 702 tasks      | elapsed:   19.4s
[Parallel(n_jobs=4)]: Done 703 tasks      | elapsed:   19.4s
[Parallel(n_jobs=4)]: Done 704 tasks      | elapsed:   19.4s
[Parallel(n_jobs=4)]: Done 705 tasks      | elapsed:   19.5s
[Parallel(n_jobs=4)]: Do

[Parallel(n_jobs=4)]: Done 826 tasks      | elapsed:   23.1s
[Parallel(n_jobs=4)]: Done 827 tasks      | elapsed:   23.2s
[Parallel(n_jobs=4)]: Done 828 tasks      | elapsed:   23.2s
[Parallel(n_jobs=4)]: Done 829 tasks      | elapsed:   23.2s
[Parallel(n_jobs=4)]: Done 830 tasks      | elapsed:   23.2s
[Parallel(n_jobs=4)]: Done 831 tasks      | elapsed:   23.2s
[Parallel(n_jobs=4)]: Done 832 tasks      | elapsed:   23.3s
[Parallel(n_jobs=4)]: Done 833 tasks      | elapsed:   23.3s
[Parallel(n_jobs=4)]: Done 834 tasks      | elapsed:   23.3s
[Parallel(n_jobs=4)]: Done 835 tasks      | elapsed:   23.3s
[Parallel(n_jobs=4)]: Done 836 tasks      | elapsed:   23.3s
[Parallel(n_jobs=4)]: Done 837 tasks      | elapsed:   23.3s
[Parallel(n_jobs=4)]: Done 838 tasks      | elapsed:   23.4s
[Parallel(n_jobs=4)]: Done 839 tasks      | elapsed:   23.4s
[Parallel(n_jobs=4)]: Done 840 tasks      | elapsed:   23.4s
[Parallel(n_jobs=4)]: Done 841 tasks      | elapsed:   23.4s
[Parallel(n_jobs=4)]: Do

[Parallel(n_jobs=4)]: Done 962 tasks      | elapsed:   27.2s
[Parallel(n_jobs=4)]: Done 963 tasks      | elapsed:   27.2s
[Parallel(n_jobs=4)]: Done 964 tasks      | elapsed:   27.3s
[Parallel(n_jobs=4)]: Done 965 tasks      | elapsed:   27.3s
[Parallel(n_jobs=4)]: Done 966 tasks      | elapsed:   27.3s
[Parallel(n_jobs=4)]: Done 967 tasks      | elapsed:   27.3s
[Parallel(n_jobs=4)]: Done 968 tasks      | elapsed:   27.3s
[Parallel(n_jobs=4)]: Done 969 tasks      | elapsed:   27.4s
[Parallel(n_jobs=4)]: Done 970 tasks      | elapsed:   27.4s
[Parallel(n_jobs=4)]: Done 971 tasks      | elapsed:   27.4s
[Parallel(n_jobs=4)]: Done 972 tasks      | elapsed:   27.4s
[Parallel(n_jobs=4)]: Done 973 tasks      | elapsed:   27.5s
[Parallel(n_jobs=4)]: Done 974 tasks      | elapsed:   27.5s
[Parallel(n_jobs=4)]: Done 975 tasks      | elapsed:   27.5s
[Parallel(n_jobs=4)]: Done 976 tasks      | elapsed:   27.6s
[Parallel(n_jobs=4)]: Done 977 tasks      | elapsed:   27.6s
[Parallel(n_jobs=4)]: Do

0.7150848558408173

## Performance of Neural Network Classifier Before Tuning

In [56]:
NN_classifier = MLPClassifier()
NN_classifier = train_classifiers(X_train,Y_train,'NN', {'verbose': 20})

Iteration 1, loss = 2.53319524
Iteration 2, loss = 1.15043572
Iteration 3, loss = 0.76161578
Iteration 4, loss = 0.55865253
Iteration 5, loss = 0.42723958
Iteration 6, loss = 0.33418717
Iteration 7, loss = 0.26639936
Iteration 8, loss = 0.21673178
Iteration 9, loss = 0.17995752
Iteration 10, loss = 0.15201532
Iteration 11, loss = 0.13043895
Iteration 12, loss = 0.11394828
Iteration 13, loss = 0.10041811
Iteration 14, loss = 0.09005231
Iteration 15, loss = 0.08100382
Iteration 16, loss = 0.07424945
Iteration 17, loss = 0.06887087
Iteration 18, loss = 0.06390743
Iteration 19, loss = 0.05991179
Iteration 20, loss = 0.05693417
Iteration 21, loss = 0.05448785
Iteration 22, loss = 0.05225749
Iteration 23, loss = 0.05046613
Iteration 24, loss = 0.04888247
Iteration 25, loss = 0.04772260
Iteration 26, loss = 0.04675358
Iteration 27, loss = 0.04590050
Iteration 28, loss = 0.04478020
Iteration 29, loss = 0.04396693
Iteration 30, loss = 0.04311005
Iteration 31, loss = 0.04291728
Iteration 32, los

In [57]:
f1_score_NN = calculate_f1score(NN_classifier,'NN',X_test,Y_test)
f1_score_NN

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.7746638906924626

# Tuning the Neural Network Classifier

In [179]:
from sklearn.model_selection import GridSearchCV# Create the parameter grid based on the results of random search 
 
from sklearn.neural_network import MLPClassifier

#hidden_layer_sizes = [100, 200, 400]
learning_rate = ['constant', 'invscaling', 'adaptive']
solver = ['lbfgs', 'sgd', 'adam']

param_tune = {'learning_rate':learning_rate, 'solver':solver}

NN = MLPClassifier()# Instantiate the grid search model

grid_search = GridSearchCV(estimator = NN, param_grid = param_tune, 
                          cv = 5, n_jobs = -1, verbose = 50)

# Fit the grid search to the data
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed: 12.3min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed: 13.5min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 22.2min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed: 23.9min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed: 23.9min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed: 24.2min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 26.3min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed: 27.0min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 28.4min
[Paralle

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_iter=200,
                                     momentum=0.9, n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_state=None, shuffle=True,
                                     solver='adam', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'learning_rate': ['constant', 'invscaling',

In [180]:
grid_search.best_params_

{'learning_rate': 'adaptive', 'solver': 'adam'}

## Performance of Neural Network Classifier After Tuning

In [58]:
NN_classifier = MLPClassifier()
NN_classifier = train_classifiers(X_train,Y_train,'NN', {'verbose': 20, 'learning_rate': 'adaptive', 'solver': 'adam'})

Iteration 1, loss = 2.57235532
Iteration 2, loss = 1.16603746
Iteration 3, loss = 0.77120634
Iteration 4, loss = 0.56677087
Iteration 5, loss = 0.43270867
Iteration 6, loss = 0.33838426
Iteration 7, loss = 0.26975279
Iteration 8, loss = 0.21923821
Iteration 9, loss = 0.18195825
Iteration 10, loss = 0.15358213
Iteration 11, loss = 0.13216352
Iteration 12, loss = 0.11510323
Iteration 13, loss = 0.10146153
Iteration 14, loss = 0.09070271
Iteration 15, loss = 0.08231323
Iteration 16, loss = 0.07513881
Iteration 17, loss = 0.06941182
Iteration 18, loss = 0.06466584
Iteration 19, loss = 0.06064808
Iteration 20, loss = 0.05775368
Iteration 21, loss = 0.05528289
Iteration 22, loss = 0.05292459
Iteration 23, loss = 0.05106105
Iteration 24, loss = 0.04923000
Iteration 25, loss = 0.04799152
Iteration 26, loss = 0.04695369
Iteration 27, loss = 0.04611178
Iteration 28, loss = 0.04464339
Iteration 29, loss = 0.04435355
Iteration 30, loss = 0.04393711
Iteration 31, loss = 0.04317972
Iteration 32, los

In [59]:
f1_score_NN = calculate_f1score(NN_classifier,'NN',X_test,Y_test)
f1_score_NN

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.7746828912266325

# Performance of Decision Tree Classifier Before Tuning

In [42]:
DT_classifier = DecisionTreeClassifier()
DT_classifier = train_classifiers(X_train,Y_train,'DT')

In [43]:
f1_score_DT = calculate_f1score(DT_classifier,'DT',X_test,Y_test)
f1_score_DT

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.6251828963904881

# Tuning the Decision Tree Classifier

In [101]:
from sklearn.model_selection import GridSearchCV# Create the parameter grid based on the results of random search 
 
from sklearn.tree import DecisionTreeClassifier
min_impurity_decrease =[0,0.1,0.2]
criterion = ['gini','entropy']
max_depth = [20,None]

param_tune = {'min_impurity_decrease':min_impurity_decrease, 'criterion':criterion, 'max_depth':max_depth}

dt = DecisionTreeClassifier()# Instantiate the grid search model

grid_search = GridSearchCV(estimator = dt, param_grid = param_tune, 
                          cv = 5, n_jobs = -1, verbose = 50)

# Fit the grid search to the data
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   15.2s
[Paralle

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [20, None],
                         'min

In [102]:
grid_search.best_params_

{'criterion': 'gini', 'max_depth': None, 'min_impurity_decrease': 0}

# Performance of Decision Tree Classifier After Tuning

In [46]:
DT_classifier = DecisionTreeClassifier()
DT_classifier = train_classifiers(X_train,Y_train,'DT',
                                  {'criterion': 'gini', 'max_depth': None, 'min_impurity_decrease': 0})

In [47]:
f1_score_DT = calculate_f1score(DT_classifier,'DT',X_test,Y_test)
f1_score_DT

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.6252524129925264

# Performance of SVM-SVC Classifier Before Tuning

In [50]:
SVC_classifier = train_classifiers(X_train,Y_train,'SVC')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [52]:
f1_score_SVC = calculate_f1score(SVC_classifier,'SVC',X_test,Y_test)
f1_score_SVC

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.07654697783178456

# Tuning the SVM SVC Classifier

In [57]:
from sklearn.model_selection import GridSearchCV# Create the parameter grid based on the results of random search 
from sklearn import svm

kernels = ['linear', 'rbf', 'poly']
gammas = [0.1, 1, 10]
C = [1, 10]

param_tune = {'kernel':kernels, 'gamma':gammas, 'C':C}

svc_lassifier = svm.SVC()

grid_search = GridSearchCV(estimator = svc_lassifier, param_grid = param_tune, cv = 5, n_jobs = -1, verbose = 50)

# Fit the grid search to the data
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   41.6s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   44.3s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   44.6s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   44.6s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  2.9min
[Paralle

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': [1, 10], 'gamma': [0.1, 1, 10],
                         'kernel': ['linear', 'rbf', 'poly']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=50)

In [59]:
grid_search.best_params_

{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}

# Performance of SVM-SVC Classifier After Tuning

In [53]:
SVC_classifier = train_classifiers(X_train,Y_train,'SVC',{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'})

In [54]:
f1_score_SVC = calculate_f1score(SVC_classifier,'SVC',X_test,Y_test)
f1_score_SVC

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.7968245135799892

# Performance of Linear Regression

In [71]:
enc = LabelEncoder()
y_enocoded = enc.fit_transform(y)

X_encoded_train,X_encoded_test,Y_encoded_train,Y_encoded_test = train_test_split(x,y_enocoded,test_size = 0.3,random_state=100) 
 
linear_classifier = train_classifiers(X_encoded_train,Y_encoded_train,"LR")

In [77]:
y_predicted = linear_classifier.predict(X_encoded_test)

rmse = mean_squared_error(Y_encoded_test, y_predicted)
r2 = r2_score(Y_encoded_test, y_predicted)
print("mean_squared_error",rmse)
print("r2_score",r2)

mean_squared_error 1219121.9092613412
r2_score -2084.9493927959265


### Reason that I did not tune the Linear Regression Classifier:

#### Linear Regression is a model used to predict discrete quantity values.
#### Classification is used for predicting the discrete class labels
#### As text classification in this assignment is based on text classification based on discrete class labels, 
#### I would like to conclude that Linear Regression is not an appropriate approach to test and train text data.



# Conclusion - Best classifier

#### After training all the above classifiers on the training dataset and then testing it on the test data, I have come to a conclusion that Support Vector Machine classifier gives the best results for text classification. 
#### Though tuning the best parameters has improved the F1 score of all the classifiers, it still lags behind the SVM-SVC classifier. 
#### The F1 score of the SVM-SVC classifier outperforms the other classifiers and is slightly higher than that of the Neural Networks. 
#### Although Neural Networks is a good approach it has a slightly lower score compared to SVM.
#### SVC classifier is better than MNB classifier when the training and testing is done on large datasets.






# REFERENCES (for all the classifiers)

#### https://www.udemy.com/course/python-data-science-machine-learning-bootcamp/
#### https://www.kaggle.com/shotashimizu/09-decisiontree-gridsearchcv
#### https://scikit-learn.org/
#### https://scikit-learn.org/stable/documentation.html